---
<center><h1>Pandas and GIS Data</h1></center>


<center><h2> Getting Spatial Data into Pandas</h2></center>

---

ArcGIS (and other GIS data) can be converted into a Pandas Dataframe.  The reasons one may want to do this is that it allows for further analysis that may not be able to be done in other GIS software, or is easier to do in the Python programming evironment.  We will cover here in this notebook several ways of accessing ingesting GIS data into a Pandas data frame.  These include:

- The Spatially Enabled Data Frame (Part of the [ArcGIS API for Python](https://developers.arcgis.com/python/guide/introduction-to-the-spatially-enabled-dataframe/));
- ArcPy - [FeatureClassToNumPyArray](https://pro.arcgis.com/en/pro-app/latest/arcpy/data-access/featureclasstonumpyarray.htm) method;

The <b>Spatially Enabled Data Frame</b> is a data type created by ESRI that extends a regular Pandas DataFrame.  It extends it in many ways that we will discuss later, but the most fundemental is that it allows you to read in ESRI vector data in a variety of different formats (local and web) into a DataFrame for further manipuation.  This is lines of code you need to import both pandas and the Spatially Enabled Data Frame.

In [32]:
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

From there you can access any local data on your system:

```python
sdf = pd.DataFrame.spatial.from_featureclass("path\to\your\data\folder\layername")
```
Here is a live example:

In [33]:
sdf = pd.DataFrame.spatial.from_featureclass("Gulf_and_Caribbean_Marine_Ecoregions.shp")
sdf

,FID,ECO_CODE,ECOREGION,PROV_CODE,PROVINCE,RLM_CODE,REALM,ALT_CODE,ECO_CODE_X,Shape_Leng,Lat_Zone,Shape_Area,SHAPE
0,0,20063,Bahamian,12,Tropical Northwestern Atlantic,4,Tropical Atlantic,66,63,2920.413576,Tropical,368517.968376,"{""rings"": [[[1018181.8780927337, 2466475.57884..."
1,1,20062,Bermuda,12,Tropical Northwestern Atlantic,4,Tropical Atlantic,67,62,1365.480635,Tropical,146257.311201,"{""rings"": [[[1170405.8802054417, 3343762.09916..."
2,2,25042,Carolinian,6,Warm Temperate Northwest Atlantic,2,Temperate Northern Atlantic,21,42,2000.205433,Temperate,222021.281976,"{""rings"": [[[318162.30004956695, 4182962.58327..."
3,3,20064,Eastern Caribbean,12,Tropical Northwestern Atlantic,4,Tropical Atlantic,68,64,2500.148984,Tropical,329291.612384,"{""rings"": [[[1578491.5565231834, 1307207.24016..."
4,4,20070,Floridian,12,Tropical Northwestern Atlantic,4,Tropical Atlantic,69,70,1214.206341,Tropical,92333.186987,"{""rings"": [[[-537064.6225060939, 3296797.47538..."
5,5,20043,Northern Gulf of Mexico,6,Warm Temperate Northwest Atlantic,2,Temperate Northern Atlantic,20,43,3340.153208,Temperate,553545.983522,"{""rings"": [[[-1014956.6566839815, 3924150.1213..."
6,6,20066,Southern Caribbean,12,Tropical Northwestern Atlantic,4,Tropical Atlantic,71,66,3329.387807,Tropical,423550.614982,"{""rings"": [[[1424155.0064206573, 1292296.63155..."
7,7,20069,Southern Gulf of Mexico,12,Tropical Northwestern Atlantic,4,Tropical Atlantic,72,69,3052.773444,Tropical,449703.392457,"{""rings"": [[[-1170089.2484145437, 2938491.3994..."
8,8,20067,Southwestern Caribbean,12,Tropical Northwestern Atlantic,4,Tropical Atlantic,73,67,2764.711942,Tropical,445707.530043,"{""rings"": [[[16022.926418437613, 1626670.31913..."
9,9,20068,Western Caribbean,12,Tropical Northwestern Atlantic,4,Tropical Atlantic,74,68,1739.869078,Tropical,153684.511701,"{""rings"": [[[-1219920.9757938143, 2528983.5536..."


In [34]:
sdf["Shape_Area"] * 0.00386102

0     1422.855246
1      564.702404
2       857.22861
3     1271.401501
4      356.500282
5     2137.252113
6     1635.337395
7     1736.313792
8     1720.885688
9      593.378973
10    1908.870111
Name: Shape_Area, dtype: Float64

Notice that there is a field called shape.  This field does contain geometry.  We will look at that later in the course (not today).

In addition to accessing local data, you can also access feature datasets on ArcGIS online, ESRI living atlas, and any data that has been published as an ArcGIS feature layer.

In [35]:
#Import a COVID data layer.  This layer contains the updated stats for each county in the United States
from arcgis.features import FeatureLayer
mylayer = FeatureLayer(("https://services1.arcgis.com/0MSEUqKaxRlEPj5g/ArcGIS/rest/services/ncov_cases_US/FeatureServer/0"))
sdf2 = pd.DataFrame.spatial.from_layer(mylayer)
sdf2

,OBJECTID,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Recovered,Deaths,Active,Admin2,FIPS,Combined_Key,Incident_Rate,People_Tested,People_Hospitalized,UID,ISO3,SHAPE
0,1,Alabama,US,2023-03-10 13:21:02,32.539527,-86.644082,19790,<NA>,232,<NA>,Autauga,01001,"Autauga, Alabama, US",35422.14824,<NA>,<NA>,84001001,USA,"{""x"": -86.64408226999996, ""y"": 32.539527450000..."
1,2,Alabama,US,2023-03-10 13:21:02,30.72775,-87.722071,69860,<NA>,727,<NA>,Baldwin,01003,"Baldwin, Alabama, US",31294.516068,<NA>,<NA>,84001003,USA,"{""x"": -87.72207057999998, ""y"": 30.727749910000..."
2,3,Alabama,US,2023-03-10 13:21:02,31.868263,-85.387129,7485,<NA>,103,<NA>,Barbour,01005,"Barbour, Alabama, US",30320.82962,<NA>,<NA>,84001005,USA,"{""x"": -85.38712859999998, ""y"": 31.868263000000..."
3,4,Alabama,US,2023-03-10 13:21:02,32.996421,-87.125115,8091,<NA>,109,<NA>,Bibb,01007,"Bibb, Alabama, US",36130.21345,<NA>,<NA>,84001007,USA,"{""x"": -87.12511459999996, ""y"": 32.996420640000..."
4,5,Alabama,US,2023-03-10 13:21:02,33.982109,-86.567906,18704,<NA>,261,<NA>,Blount,01009,"Blount, Alabama, US",32345.311797,<NA>,<NA>,84001009,USA,"{""x"": -86.56790592999994, ""y"": 33.982109180000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3267,3268,Wyoming,US,2023-03-10 13:21:02,43.935225,-110.58908,12134,<NA>,16,<NA>,Teton,56039,"Teton, Wyoming, US",51713.262871,<NA>,<NA>,84056039,USA,"{""x"": -110.58908009999999, ""y"": 43.93522482000..."
3268,3269,Wyoming,US,2023-03-10 13:21:02,41.287818,-110.547578,6406,<NA>,43,<NA>,Uinta,56041,"Uinta, Wyoming, US",31672.105211,<NA>,<NA>,84056041,USA,"{""x"": -110.54757819999998, ""y"": 41.28781830000..."
3269,3270,Wyoming,US,2023-01-08 23:21:00,<NA>,<NA>,0,<NA>,0,<NA>,Unassigned,90056,"Unassigned, Wyoming, US",<NA>,<NA>,<NA>,84090056,USA,None
3270,3271,Wyoming,US,2023-03-10 13:21:02,43.904516,-107.680187,2755,<NA>,51,<NA>,Washakie,56043,"Washakie, Wyoming, US",35297.885971,<NA>,<NA>,84056043,USA,"{""x"": -107.68018699999999, ""y"": 43.90451606000..."


In [36]:
sdf2[sdf2["Province_State"] == 'Mississippi']['Confirmed'].sum()

990756

In [37]:
# Marine protected areas of the world
from arcgis.features import FeatureLayer
mylayer = FeatureLayer(("https://services5.arcgis.com/Mj0hjvkNtV7NRhA7/arcgis/rest/services/WDPA_v4/FeatureServer/0"))
sdf3 = pd.DataFrame.spatial.from_layer(mylayer)
sdf3

,OBJECTID,wdpaid,wdpa_pid,pa_def,name,orig_name,desig,desig_eng,desig_type,iucn_cat,int_crit,marine,rep_m_area,rep_area,no_take,no_tk_area,status,status_yr,gov_type,own_type,mang_auth,mang_plan,verif,metadataid,sub_loc,parent_iso3,iso3,SHAPE
0,1,159863.0,159863,1,Ostriv,Ostriv,Regional Zakaznik,Regional Zakaznik,National,IV,Not Applicable,0,0.0,0.23,Not Applicable,0.0,Designated,1995,Not Reported,Not Reported,Not Reported,Not Reported,State Verified,367,UA-35,UKR,UKR,"{""points"": [[3486495.5048, 6227484.1114]], ""sp..."
1,2,160202.0,160202,1,Bilka,Bilka,Regional Zakaznik,Regional Zakaznik,National,IV,Not Applicable,0,0.0,0.2,Not Applicable,0.0,Designated,1994,Not Reported,Not Reported,Not Reported,Not Reported,State Verified,367,UA-77,UKR,UKR,"{""points"": [[2856458.1338, 6136852.834]], ""spa..."
2,3,555681859.0,555681859,1,Bamézoun,Bamézoun,Forêt Sacrée,Sacred Forest,Not Applicable,II,Not Applicable,0,0.0,0.47,Not Applicable,0.0,Established,1915,Indigenous peoples,Communal,Local management committee Bamézoun,Not Reported,Expert Verified,1980,Not Reported,BEN,BEN,"{""points"": [[283757.834800001, 723412.8816]], ..."
3,4,555681858.0,555681858,1,Orozoun,Orozoun,Forêt Sacrée,Sacred Forest,Not Applicable,II,Not Applicable,0,0.0,0.0161,Not Applicable,0.0,Established,1915,Indigenous peoples,Communal,Local management committee,Not Reported,Expert Verified,1980,Not Reported,BEN,BEN,"{""points"": [[301116.996199999, 724717.22309999..."
4,5,555681857.0,555681857,1,Têdozoun,Têdozoun,Forêt Sacrée,Sacred Forest,Not Applicable,II,Not Applicable,0,0.0,0.104136,Not Applicable,0.0,Established,1951,Indigenous peoples,Communal,Local management committee,Not Reported,Expert Verified,1980,Not Reported,BEN,BEN,"{""points"": [[239421.508000001, 743421.65630000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13932,11933,154167.0,154167,1,2012013 Ek,2012013 Ek,Naturminne,Natural Monument,National,Not Assigned,Not Applicable,0,0.0,0.0,Not Applicable,0.0,Designated,1939,Federal or national ministry or agency,Not Reported,County Administrative Board and Local Authorities,Not Reported,State Verified,2013,Not Reported,SWE,SWE,"{""points"": [[1724294.5761, 7655718.2255]], ""sp..."
13933,11934,182519.0,182519,1,Rrepet e Izvorit,Rrepet e Izvorit,Monument Natyre,Nature Monument (Category III IUCN),National,III,Not Applicable,0,0.0,0.0,Not Applicable,0.0,Designated,2002,Federal or national ministry or agency,Not Reported,"Ministry of Environment, Forests and Water Adm...",Not Reported,State Verified,2013,Not Reported,ALB,ALB,"{""points"": [[2171535.8066, 4914170.3959]], ""sp..."
13934,11935,171451.0,171451,1,Apollo kivi e Kalevipoja viskekivi,Apollo kivi e Kalevipoja viskekivi,Kaitstav looduse üksikobjekt,Protected nature monument,National,III,Not Applicable,0,0.0,0.0,Not Applicable,0.0,Designated,2023,Federal or national ministry or agency,Not Reported,"Ministry of Environment, Narva mnt 7a, Tallinn...",Not Reported,State Verified,2013,Not Reported,EST,EST,"{""points"": [[3019674.941, 8098704.0328]], ""spa..."
13935,11936,387130.0,387130,1,Planica - lipa pri lovskem domu - dendrološki ...,Planica - lipa pri lovskem domu - dendrološki ...,naravni spomenik,Natural Monument,National,III,Not Applicable,0,0.0,0.0,Not Applicable,0.0,Designated,1992,Federal or national ministry or agency,Not Reported,"Ministry of Agriculture and Environment, Dunaj...",Not Reported,State Verified,2013,Not Reported,SVN,SVN,"{""points"": [[1734274.7379, 5855694.8217]], ""sp..."


You can also use the two arcPy methods called "FeatureClassToNumPyArray" and "TableToNumPyArray"  For each of these methods, the inital result is an array instead of a pandas data frame.  To run these methods, you must specify the location of the local data, and the fields you want to convert, then convert that into a pandas DataFrame:

In [38]:
import arcpy
fields = ["Shape_Area", "ECOREGION", "REALM"]
array = arcpy.da.FeatureClassToNumPyArray("Gulf_and_Caribbean_Marine_Ecoregions.shp", fields)
array

sdf4 = pd.DataFrame(array[fields])
sdf4

,Shape_Area,ECOREGION,REALM
0,368517.968376,Bahamian,Tropical Atlantic
1,146257.311201,Bermuda,Tropical Atlantic
2,222021.281976,Carolinian,Temperate Northern Atlantic
3,329291.612384,Eastern Caribbean,Tropical Atlantic
4,92333.186987,Floridian,Tropical Atlantic
5,553545.983522,Northern Gulf of Mexico,Temperate Northern Atlantic
6,423550.614982,Southern Caribbean,Tropical Atlantic
7,449703.392457,Southern Gulf of Mexico,Tropical Atlantic
8,445707.530043,Southwestern Caribbean,Tropical Atlantic
9,153684.511701,Western Caribbean,Tropical Atlantic


In [39]:
import arcpy
fields = ["Crosswalk", "Class", "Match"]
array = arcpy.da.FeatureClassToNumPyArray("aassesmet.shp", fields)
array

sdf5 = pd.DataFrame(array[fields])
sdf5

,Crosswalk,Class,Match
0,Sand,Sand,1
1,Coral/Algae,Coral/Algae,1
2,Coral/Algae,Coral/Algae,1
3,Hardbottom Dense Algae,Hardbottom Dense Algae,1
4,Hardbottom Dense Algae,Hardbottom Dense Algae,1
...,...,...,...
2681,Sand,Sand,1
2682,Sand,Sand,1
2683,Hardbottom Dense Algae,Hardbottom Dense Algae,1
2684,Coral/Algae,Coral/Algae,1


In [40]:
sdf5["Crosswalk"] == sdf5["Class"]

0       True
1       True
2       True
3       True
4       True
        ... 
2681    True
2682    True
2683    True
2684    True
2685    True
Length: 2686, dtype: bool

In [43]:
sdf5["Match"] = sdf5["Crosswalk"] == sdf5["Class"]
sdf5

,Crosswalk,Class,Match
0,Sand,Sand,True
1,Coral/Algae,Coral/Algae,True
2,Coral/Algae,Coral/Algae,True
3,Hardbottom Dense Algae,Hardbottom Dense Algae,True
4,Hardbottom Dense Algae,Hardbottom Dense Algae,True
...,...,...,...
2681,Sand,Sand,True
2682,Sand,Sand,True
2683,Hardbottom Dense Algae,Hardbottom Dense Algae,True
2684,Coral/Algae,Coral/Algae,True


672